In [1]:
import ee
import geemap
import json
import os
#from geemap import geojson_to_ee
from ipyleaflet import GeoJSON
import segment_attributes_func as gsf

In [4]:
ee.Authenticate()

True

In [2]:
#Initializing the ee project

ee.Initialize(project = 'ee-jdawsey')

### Pulling data

In [3]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
shp_folder_path = 'E:/!!Research/!!!Data/shrubmap_modification_tests/polygon_shp'
shp_file = '/square.shp'
shp_area = geemap.shp_to_ee(f'{shp_folder_path}{shp_file}')

In [5]:
polygon_imagery = gsf.get_imagery(shp_area, 'USDA/NAIP/DOQQ', start_filter_date='2016-01-01', end_filter_date='2016-12-31')

In [6]:
band_maths = {'savi' : '((1 + 0.6) * (b("N") - b("R"))) / (b("N") + b("R") + 0.5)',
              'endvi' : '((b("N") + b("G")) - (2 * b("B"))) / ((b("N") + b("G")) + (2 + b("B")))'}
preprocessed = gsf.preprocess_image(polygon_imagery, given_scale = 1, given_region = shp_area, band_expressions=band_maths)

savi ((1 + 0.6) * (b("N") - b("R"))) / (b("N") + b("R") + 0.5)
endvi ((b("N") + b("G")) - (2 * b("B"))) / ((b("N") + b("G")) + (2 + b("B")))


In [7]:
#preprocessed = preprocessed.clip(shp_area.geometry())
viz_params = {
    'bands': ['R', 'G', 'B'], # Adjust to match your bands
    'min' : -1.5,
    'max' : 2,
}
Map.addLayer(preprocessed, viz_params)

In [10]:
geemap.ee_export_image(
    preprocessed, filename=f'{shp_folder_path}/preprocessed.tif', scale=2, region=shp_area.geometry(), file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to E:\!!Research\!!!Data\shrubmap_modification_tests\polygon_shp\preprocessed.tif


In [7]:
segmented = gsf.segmentation_attributes(preprocessed)

Original bands: ['R', 'G', 'B', 'N', 'savi', 'endvi']
Mean bands list: ['R_mean', 'G_mean', 'B_mean', 'N_mean', 'savi_mean', 'endvi_mean']
Mean bands list with clusters: ['R_mean', 'G_mean', 'B_mean', 'N_mean', 'savi_mean', 'endvi_mean', 'clusters']
['R', 'G', 'B', 'N', 'savi', 'endvi', 'R_1', 'G_1', 'B_1', 'N_1', 'savi_1', 'endvi_1', 'area', 'perimeter', 'width', 'height']


In [8]:
print(segmented.bandNames().getInfo())

['R', 'G', 'B', 'N', 'savi', 'endvi', 'R_1', 'G_1', 'B_1', 'N_1', 'savi_1', 'endvi_1', 'area', 'perimeter', 'width', 'height']


In [10]:
# Define visualization parameters
viz_params = {
    'bands': ['N_1', 'G_1', 'B_1'], # Adjust to match your bands
    'min' : -1.5,
    'max' : 2,
}
# Add the segmented image to the map
Map.addLayer(segmented, viz_params, 'Segmented Image')

In [8]:
#from my_gee_functions import stdrd_ref_func

# if pulling from a buffer, set the polygon number
poly_num = 22

#specify the max number of clusters for the xmeans algorithm
max_clust = 15

#specify the area
shp = ee_items[poly_num]

# image name for export
image_name = f'poly{poly_num}'
#image_export_directory = shp_folder_path

# then specify the geometry of the item wanting to be exported
shp_geom = shp.geometry()


# standardizing the imagery
standardized = mgf.stdrd_ref_func(shp_geom)

# viewing on the map
vis_param_gauss = {'bands' : ['endvi', 'R', 'G'], 
               'min' : -1.5, 
               'max' : 2,
               'gamma' : 1}
Map.addLayer(standardized, vis_param_gauss, f'poly{poly_num}')

# setting up the parameters to run the segmentation algorithm
segmented_image = mgf.segmentation_func(standardized)